In [ ]:
# integrate pytorch with Bayesian Optimization to potentially improve results - not working yet :( 

In [62]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer, AdamOptimizer
from pennylane.templates import AngleEmbedding, StronglyEntanglingLayers
import matplotlib.pyplot as plt
from skopt import gp_minimize
from skopt.space import Integer, Real
from sklearn.cluster import KMeans
import torch
import torch.nn as nn
import torch.optim as optim
import random

In [63]:
dev = qml.device("default.qubit")

In [64]:
# angle encoding - maps data to rotation angles for quantum gates.
def get_angles(x):
    beta0 = 2 * np.arcsin(np.sqrt(x[1] ** 2) / np.sqrt(x[0] ** 2 + x[1] ** 2 + 1e-12))
    beta1 = 2 * np.arcsin(np.sqrt(x[3] ** 2) / np.sqrt(x[2] ** 2 + x[3] ** 2 + 1e-12))
    beta2 = 2 * np.arcsin(np.linalg.norm(x[2:]) / np.linalg.norm(x))

    return np.array([beta2, -beta1 / 2, beta1 / 2, -beta0 / 2, beta0 / 2])
    # return np.array([beta2, -beta1 / 2, beta1 / 2, -beta0 / 2, beta0 / 2], dtype=np.float32)
    # return torch.tensor([beta2, -beta1 / 2, beta1 / 2, -beta0 / 2, beta0 / 2], dtype=torch.float32)

In [87]:
# LOAD MATLAB DATA
# import scipy.io
# mat = scipy.io.loadmat('/Users/trisha/Documents/CSIRE/SMIB/SMIB_feature.mat')
# X = mat["X_train"]
# Y = mat["Y_train"]

# X = np.array(X)
# Y = np.array(Y)

# # write to a file
# np.savetxt("X_train.txt", X)
# np.savetxt("Y_train.txt", Y)

In [88]:
# LOAD .TXT DATA
data = np.loadtxt("trainX.txt")
X = data[:, 0:2]
Y = np.loadtxt("trainY.txt")

In [89]:
# process data - convert to tensor
Y = torch.tensor(np.where(Y == 0, -1.0, 1.0), dtype=torch.float32)
# print(Y)

padding = np.ones((len(X), 2)) * 0.1
X_pad = np.c_[X, padding]
normalization = np.sqrt(np.sum(X_pad**2, -1))
X_norm = (X_pad.T / normalization).T
features = np.array([get_angles(x) for x in X_norm], requires_grad=False)
features = torch.tensor(features, dtype=torch.float32)
# print(features)

# Split data into training and validation sets
np.random.seed(0)
num_data = len(Y)
num_train = int(0.75 * num_data) # 75% of the data for training
index = np.random.permutation(num_data) # permutation of integers 0 -> len(Y) (indices)
# print(index)

feats_train = features[index[:num_train]]
feats_val = features[index[num_train:]]

Y_train = Y[index[:num_train]]
Y_val = Y[index[num_train:]]

In [90]:
# CIRCUIT + COST FUNCTIONS
def state_preparation(a):
    qml.RY(a[0], wires=0)

    qml.CNOT(wires=[0, 1])
    qml.RY(a[1], wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RY(a[2], wires=1)

    qml.PauliX(wires=0)
    qml.CNOT(wires=[0, 1])
    qml.RY(a[3], wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RY(a[4], wires=1)
    qml.PauliX(wires=0)

def layer(layer_weights):
    for wire in range(num_qubits):
        qml.Rot(*layer_weights[wire], wires=wire)
    qml.CNOT(wires=[0, 1])
    for i in range(num_qubits): 
        qml.CZ(wires=[i, (i+1) % num_qubits]) 
    for i in range(num_qubits): 
        qml.S(wires=i)  

@qml.qnode(dev, interface='torch') # added interface = torch
def circuit(weights, x):
    state_preparation(x)
    for wire in range(num_qubits):
        qml.Hadamard(wires=wire)
    for layer_weights in weights:
        layer(layer_weights)
    return qml.expval(qml.PauliZ(0))

def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

def square_loss(labels, predictions):
    # return np.mean((labels - qml.math.stack(predictions)) ** 2)
    # return torch.mean((labels - predictions) ** 2)

    # print("labels: ", labels)
    # print("predictions: ", predictions)

    labels = labels.float()  # Convert to float if necessary
    predictions = predictions.float()  # Convert to float if necessary
    
    # Compute mean squared error
    mse = torch.mean((labels - predictions) ** 2)
    return mse

def accuracy(labels, predictions):
    acc = sum(abs(l - p) < 1e-5 for l, p in zip(labels, predictions))
    return acc / len(labels)

def cost(weights, bias, X, Y):
    predictions = variational_classifier(weights, bias, X.T)
    return square_loss(Y, predictions)

In [91]:
# Define the objective function for Bayesian optimization
def objective(params):
    global num_qubits
    # num_qubits = 2
    num_qubits, num_layers, learning_rate, opt_steps = params
    opt_steps = int(opt_steps)
    num_qubits = int(num_qubits)
    num_layers = int(num_layers)

    print("num_qubits: ", num_qubits, " num layers: ", num_layers, " learning rate: ", learning_rate, " opt_steps: ", opt_steps)

    # Initialize variables
    # weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
    # bias_init = np.array(0.0, requires_grad=True) 
    # opt = AdamOptimizer(learning_rate)
    
    # torch variables
    # weights_init = 0.01 * torch.randn(num_layers, num_qubits, 3, requires_grad=True)
    # bias_init = torch.tensor(0.0, requires_grad=True)
    weights_init = torch.nn.Parameter(0.01 * torch.randn(num_layers, num_qubits, 3))
    bias_init = torch.nn.Parameter(torch.tensor(0.0))

    optimizer = optim.Adam([weights_init, bias_init], lr=learning_rate)
    
    batch_size = 128

    # Train the variational classifier
    weights = weights_init
    bias = bias_init
    acc_val = 0
    for it in range(opt_steps):
        batch_index = np.random.randint(0, num_train, (batch_size,)) # array of random integers 
        feats_train_batch = feats_train[batch_index]
        Y_train_batch = Y_train[batch_index]
        
        # weights, bias, _, _ = opt.step(cost, weights, bias, feats_train_batch, clusters_train_batch)

        # tensor opt step
        optimizer.zero_grad()
        loss = cost(weights, bias, feats_train_batch, Y_train_batch)
        loss.backward()
        optimizer.step()

        # Compute predictions on train and validation set
        # predictions_train = np.sign(variational_classifier(weights, bias, feats_train.T))
        # predictions_val = np.sign(variational_classifier(weights, bias, feats_val.T))
        
        predictions_train = torch.sign(variational_classifier(weights, bias, feats_train.T))
        predictions_val = torch.sign(variational_classifier(weights, bias, feats_val.T))

        # Compute accuracy on train and validation set
        acc_train = accuracy(Y_train, predictions_train)
        acc_val = accuracy(Y_val, predictions_val)

        if (it + 1) % opt_steps == 0:
            _cost = cost(weights, bias, features, Y)
            print(
                f"Iter: {it + 1:5d} | Cost: {_cost:0.7f} | "
                f"Acc train: {acc_train:0.7f} | Acc validation: {acc_val:0.7f}"
            )

    # Compute predictions on validation set
    # predictions_val = np.sign(variational_classifier(weights, bias, feats_val.T))
    # acc_val = accuracy(clusters_val, predictions_val)
    # print("accuracy: ", acc_val)
    
    # Ensure that acc_val is a scalar
    acc_val = float(acc_val)

    # Return the negative validation accuracy
    return -acc_val

In [93]:
# Define the search space
space  = [Integer(2, 6, name='num_qubits'), Integer(2, 6, name='num_layers'), Real(0.01, 0.1, name='learning_rate'), Integer(5, 20, name='opt_steps')]

# Run Bayesian optimization
res_gp = gp_minimize(objective, space, n_calls=15, random_state=0)

print(f"Best accuracy: {-res_gp.fun}") 
print(f"Best parameters: {res_gp.x}")

num_qubits:  4  num layers:  5  learning rate:  0.08721510558604813  opt_steps:  18
Iter:    18 | Cost: 0.8794990 | Acc train: 0.6183333 | Acc validation: 0.6175000
num_qubits:  4  num layers:  4  learning rate:  0.036778114589002514  opt_steps:  6
Iter:     6 | Cost: 1.0149202 | Acc train: 0.5583333 | Acc validation: 0.5700000
num_qubits:  3  num layers:  4  learning rate:  0.08309518558979441  opt_steps:  12
Iter:    12 | Cost: 0.8926169 | Acc train: 0.5808333 | Acc validation: 0.5675000
num_qubits:  4  num layers:  5  learning rate:  0.04036565443755416  opt_steps:  15
Iter:    15 | Cost: 0.9083252 | Acc train: 0.6066667 | Acc validation: 0.6100000
num_qubits:  3  num layers:  6  learning rate:  0.02263157023713807  opt_steps:  18
Iter:    18 | Cost: 0.9726591 | Acc train: 0.6641667 | Acc validation: 0.6900000
num_qubits:  4  num layers:  5  learning rate:  0.05684297315960845  opt_steps:  15
Iter:    15 | Cost: 0.8841655 | Acc train: 0.7066666 | Acc validation: 0.6975000
num_qubits